# Chatbot using NLP and Neural Networks in Python

Tag means classes

Patterns means what user is going to ask

Response is chatbot response

In [1]:
data={'intents':[
    {'tag':'greeting',
     'patterns':['Hello','How are you?','Hi There','Hi',"What's up"],
     'responses':['Howdy Partner!','Hello','How are you doing?','Greetings!','How do you do']
    },
    {'tag':'age',
     'patterns':['how old are you','when is your birthday','when was you born'],
     'responses':['I am 24 years old','I was born in 1966','My birthday is July 3rd and I was born in 1996','03/07/1996']
    },
    {'tag':'date',
     'patterns':['what are you doing this weekend','do you want to hangout sometime?','what are your plans for this week'],
     'responses':['I am available this week',"I don't have any plans",'I am not busy']
    },
    {'tag':'name',
     'patterns':["what is you name",'what are you called','who are you'],
      'responses':['My name is Kippi',"I'm Kippi",'Kippi']
    },
    {'tag':'goodbye',
     'patterns':['bye','g2g','see ya','adios','cya'],
     'responses':['It was nice speaking to you','See you later','Speak soon']
    },
]}

For each tag we created, we would specify patterns. Essentially this defines the different ways of how a user may pose a query to the chatbot.

The chatbot would then take these patterns and use them as training data to determine what someone is asking and the chatbot response would be relevant to that qustion.

In [2]:
import json
import string
import random

import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout

nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In order to create our training data below steps to be followed

Create a vocabulary of all the words used in the patterns

Create a list of the classes- tag of each intent

Create a list of all the patterns within the intents file

Create a list of all the associated tags to go with each patterns in the intents file.

Initialising lemmatier to get stem of words

In [3]:
lemmatizer=WordNetLemmatizer()

words=[]
classes=[]
doc_x=[]
doc_y=[]

Loop through all the intents

Tokenize each pattern and append token to words, the patterns and the associated tag to their associated list

In [4]:
for intent in data['intents']:
    for pattern in intent['patterns']:
        tokens=nltk.word_tokenize(pattern)
        words.extend(tokens)
        doc_x.append(pattern)
        doc_y.append(intent['tag'])
    if intent['tag'] not in classes:
        classes.append(intent['tag'])

Lemmatize all the words in the vocab and convert them to lowercase

In [5]:
word=[lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]

Sorting the vocab and classes in alphabetical order and taking the set to ensure no duplicates occur

In [6]:
words=sorted(set(words))
classes=sorted(set(classes))

In [7]:
print(words)

["'s", '?', 'Hello', 'Hi', 'How', 'There', 'What', 'adios', 'are', 'birthday', 'born', 'bye', 'called', 'cya', 'do', 'doing', 'for', 'g2g', 'hangout', 'how', 'is', 'name', 'old', 'plans', 'see', 'sometime', 'this', 'to', 'up', 'want', 'was', 'week', 'weekend', 'what', 'when', 'who', 'ya', 'you', 'your']


In [8]:
print(classes)

['age', 'date', 'goodbye', 'greeting', 'name']


In [9]:
print(doc_x)

['Hello', 'How are you?', 'Hi There', 'Hi', "What's up", 'how old are you', 'when is your birthday', 'when was you born', 'what are you doing this weekend', 'do you want to hangout sometime?', 'what are your plans for this week', 'what is you name', 'what are you called', 'who are you', 'bye', 'g2g', 'see ya', 'adios', 'cya']


In [10]:
print(doc_y)

['greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'age', 'age', 'age', 'date', 'date', 'date', 'name', 'name', 'name', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye']


In [11]:
training=[]
out_empty=[0]*len(classes)

for idx,doc in enumerate(doc_x):
    bow=[]
    text=lemmatizer.lemmatize(doc.lower())
    for word in words:
        bow.append(1) if word in text else bow.append(0)
    output_row=list(out_empty)
    output_row[classes.index(doc_y[idx])]=1
    training.append([bow,output_row])
random.shuffle(training)

training=np.array(training,dtype=object)
train_X=np.array(list(training[:,0]))
train_y=np.array(list(training[:,1]))

In [12]:
input_shape=(len(train_X[0]),)
output_shape=len(train_y[0])

epochs=500

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout

model=Sequential()
model.add(Dense(128,input_shape=input_shape,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(output_shape,activation='softmax'))

adam=tf.keras.optimizers.Adam(learning_rate=0.01)

model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])

print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               5120      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 5)                 325       
                                                                 
Total params: 13701 (53.52 KB)
Trainable params: 13701 (53.52 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [14]:
model.fit(x=train_X,y=train_y,epochs=500,verbose=1)

Epoch 1/500


1/1 [==============================] - 7s 7s/step - loss: 1.5306 - accuracy: 0.2632
Epoch 2/500
1/1 [==============================] - 0s 15ms/step - loss: 1.5563 - accuracy: 0.3158
Epoch 3/500
1/1 [==============================] - 0s 16ms/step - loss: 1.4178 - accuracy: 0.4211
Epoch 4/500
1/1 [==============================] - 0s 10ms/step - loss: 1.3742 - accuracy: 0.5789
Epoch 5/500
1/1 [==============================] - 0s 16ms/step - loss: 1.1994 - accuracy: 0.6842
Epoch 6/500
1/1 [==============================] - 0s 18ms/step - loss: 1.0616 - accuracy: 0.8421
Epoch 7/500
1/1 [==============================] - 0s 8ms/step - loss: 0.9520 - accuracy: 0.9474
Epoch 8/500
1/1 [==============================] - 0s 16ms/step - loss: 0.9110 - accuracy: 0.7895
Epoch 9/500
1/1 [==============================] - 0s 12ms/step - loss: 0.8340 - accuracy: 0.8947
Epoch 10/500
1/1 [==============================] - 0s 13ms/step - loss: 0.7553 - accuracy: 0.7368
Epoch 11/500
1/1 [==

In [15]:
def clean_text(text):
    tokens=nltk.word_tokenize(text)
    tokens=[lemmatizer.lemmatize(word) for word in tokens]
    return tokens

def bag_of_words(text,vocab):
    tokens=clean_text(text)
    bow=[0]*len(vocab)
    for w in tokens:
        for idx,word in enumerate(vocab):
            if word ==w:
                bow[idx]=1
    return np.array(bow)

In [16]:
def pred_class(text,vocab,labels):
    bow=bag_of_words(text,vocab)
    result=model.predict(np.array([bow]))[0]
    thresh=0.2
    y_pred=[[idx,res] for idx,res in enumerate(result) if res>thresh]
    y_pred.sort(key=lambda x:x[1],reverse=True)
    return_list=[]
    for r in y_pred:
        return_list.append(labels[r[0]])
    return return_list

def get_response(intents_list,intents_json):
    tag=intents_list[0]
    list_of_intents=intents_json['intents']
    for i in list_of_intents:
        if i['tag']==tag:
            result=random.choice(i['responses'])
            break
    return result

Running the chatbot

In [ ]:
while True:
    message=input("")
    intents=pred_class(message,words,classes)
    result=get_response(intents,data)
    print(result)

 Hi


1/1 [==============================] - 1s 658ms/step
How are you doing?


 when was you born


1/1 [==============================] - 0s 169ms/step
I am 24 years old


 what are you doing this weekend


1/1 [==============================] - 0s 136ms/step
I am not busy


 what is your name


1/1 [==============================] - 0s 126ms/step
My name is Kippi


 bye


1/1 [==============================] - 0s 174ms/step
See you later
